<a href="https://colab.research.google.com/github/deepika-3/Deepa/blob/main/Tibia_Masks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
import zipfile
import numpy as np
import nibabel as nib
from scipy.ndimage import binary_dilation, gaussian_filter, map_coordinates
from google.colab import files  # Remove if not on Colab

# Create output directory
output_dir = "output_masks"
os.makedirs(output_dir, exist_ok=True)

# Simulate tibia mask (simple cube for demo),np.zero is used to create background or no mask,later tibia region is
# represented.
mask = np.zeros((128, 128, 128), dtype=np.uint8)
mask[40:90, 40:90, 40:90] = 1  # Simulated "tibia" region

# Save function ,this function takes 3d mask array and converts it into Nifti form
# and saves in to output file.

def save_mask(mask_array, filename):
    img = nib.Nifti1Image(mask_array.astype(np.uint8), affine=np.eye(4))
    nib.save(img, os.path.join(output_dir, filename + ".nii.gz"))

# Elastic deformation function
def elastic_deform(mask, seed, alpha=10, sigma=4):
    np.random.seed(seed)
    shape = mask.shape

    dx = gaussian_filter((np.random.rand(*shape) * 2 - 1), sigma, mode="constant") * alpha
    dy = gaussian_filter((np.random.rand(*shape) * 2 - 1), sigma, mode="constant") * alpha
    dz = gaussian_filter((np.random.rand(*shape) * 2 - 1), sigma, mode="constant") * alpha

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]), indexing="ij")
    indices = np.reshape(x + dx, (-1, 1)), np.reshape(y + dy, (-1, 1)), np.reshape(z + dz, (-1, 1))

    deformed = map_coordinates(mask, indices, order=1, mode='reflect').reshape(shape)
    return (deformed > 0.5).astype(np.uint8)

# Masks to generate with parameters,takes parameters
tasks = [
    ("Original_Mask", mask),
    ("Expanded_2mm_Mask", binary_dilation(mask, iterations=3)),
    ("Expanded_4mm_Mask", binary_dilation(mask, iterations=6)),
]

# Add randomized masks with different seeds, these numbers are seeds,which reproduce defferent
#but consistent random transformations.

random_seeds = [42, 99]
for i, seed in enumerate(random_seeds, start=1):
    randomized_mask = elastic_deform(mask, seed=seed)
    tasks.append((f"Randomized_Mask_{i}", randomized_mask))

# Saving all masks
for name, mask_data in tasks:
    print(f"Saving {name}...")
    save_mask(mask_data, name)

# Zip all saved masks
zip_filename = "Tibia_Masks.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in os.listdir(output_dir):
        zipf.write(os.path.join(output_dir, file), arcname=file)

print(f"All masks saved and zipped as {zip_filename}")

# Download zip in Google Colab
files.download(zip_filename)


Saving Original_Mask...
Saving Expanded_2mm_Mask...
Saving Expanded_4mm_Mask...
Saving Randomized_Mask_1...
Saving Randomized_Mask_2...
All masks saved and zipped as Tibia_Masks.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>